In [ ]:
from aws_cdk import (
    Stack,
    RemovalPolicy,
    aws_s3 as s3,
    aws_ec2 as ec2,
    aws_iam as iam,
    aws_ecr_assets as ecr_assets
)
from constructs import Construct
import os
from pathlib import Path
from shared_stack.lambda_trigger import BatchSubmitterLambdaConstruct
from aws_cdk import aws_secretsmanager as secretsmanager
from aws_cdk import SecretValue


from shared_stack.shared_components.aws_batch_stack import FargateBatchSetup

class EnvStack(Stack):

    def __init__(self, scope: Construct, construct_id: str, **kwargs) -> None:
        super().__init__(scope, construct_id, **kwargs)
        env_type = self.node.try_get_context("env_type") or 'staging'

        # S3 bucket
        bucket = s3.Bucket(
            self, 
            "BlinkMLBucket",
            removal_policy=RemovalPolicy.DESTROY,
            auto_delete_objects=True
        )

        # Store bucket name in Secrets Manager
        secretsmanager.Secret(
            self,
            "BlinkMLBucketSecret",
            secret_name="blinkgeneralml/s3/bucket-name",
            description="Contains the Blink General ML S3 bucket name",
            secret_object_value={
                "bucket_name": SecretValue.unsafe_plain_text(bucket.bucket_name)

            }
        )
        # VPC with isolated subnets (no public IPs)
        vpc = ec2.Vpc(
            self, 
            "BlinkMLVpc",
            max_azs=2,
            nat_gateways=1
        )

        # IAM role for EC2 with SSM permissions
        role = iam.Role(
            self, "SSMInstanceRole",
            assumed_by=iam.ServicePrincipal("ec2.amazonaws.com")
        )

        role.add_managed_policy(
            iam.ManagedPolicy.from_aws_managed_policy_name("AmazonSSMManagedInstanceCore")
        )

        role.add_managed_policy(
            iam.ManagedPolicy.from_aws_managed_policy_name("AmazonEC2FullAccess")
        )

        role.add_managed_policy(
            iam.ManagedPolicy.from_aws_managed_policy_name("AmazonEventBridgeFullAccess")
        )

        role.add_managed_policy(
            iam.ManagedPolicy.from_aws_managed_policy_name("AmazonS3FullAccess")
        )

        role.add_managed_policy(
            iam.ManagedPolicy.from_aws_managed_policy_name("AmazonSSMFullAccess")
        )

        role.add_managed_policy(
            iam.ManagedPolicy.from_aws_managed_policy_name("AWSBatchFullAccess")
        )

        role.add_managed_policy(
            iam.ManagedPolicy.from_aws_managed_policy_name("AWSLambda_FullAccess")
        )

        role.add_managed_policy(
            iam.ManagedPolicy.from_aws_managed_policy_name("SecretsManagerReadWrite")
        )
        # Define the inline policy
        custom_policy = iam.Policy(self, "CustomECRCloudFormationPolicy",
            statements=[
                iam.PolicyStatement(
                    sid="ECRAccess",
                    effect=iam.Effect.ALLOW,
                    actions=[
                        "ecr:CreateRepository",
                        "ecr:DescribeRepositories",
                        "ecr:GetAuthorizationToken",
                        "ecr:SetRepositoryPolicy",
                        "ecr:TagResource",
                        "ecr:PutLifecyclePolicy",
                        "ecr:DeleteRepository"
                    ],
                    resources=["arn:aws:ecr:ap-southeast-1:056198491146:repository/*"]
                ),
                iam.PolicyStatement(
                    sid="SSMParameterStore",
                    effect=iam.Effect.ALLOW,
                    actions=[
                        "ssm:PutParameter",
                        "ssm:GetParameter",
                        "ssm:DeleteParameter"
                    ],
                    resources=["arn:aws:ssm:ap-southeast-1:056198491146:parameter/*"]
                ),
                iam.PolicyStatement(
                    sid="S3AccessForAssets",
                    effect=iam.Effect.ALLOW,
                    actions=[
                        "s3:CreateBucket",
                        "s3:PutObject",
                        "s3:GetObject",
                        "s3:ListBucket",
                        "s3:DeleteObject"
                    ],
                    resources=[
                        "arn:aws:s3:::cdktoolkit-stagingbucket-*",
                        "arn:aws:s3:::cdktoolkit-stagingbucket-*/*"
                    ]
                ),
                iam.PolicyStatement(
                    sid="CloudFormationPermissions",
                    effect=iam.Effect.ALLOW,
                    actions=[
                        "cloudformation:CreateStack",
                        "cloudformation:DescribeStacks",
                        "cloudformation:UpdateStack",
                        "cloudformation:DeleteStack",
                        "cloudformation:GetTemplate",
                        "cloudformation:ValidateTemplate"
                    ],
                    resources=["*"]
                ),
                iam.PolicyStatement(
                    sid="IAMForBootstrapRoles",
                    effect=iam.Effect.ALLOW,
                    actions=[
                        "iam:GetRole",
                        "iam:PassRole",
                        "iam:CreateRole",
                        "iam:AttachRolePolicy",
                        "iam:DetachRolePolicy",
                        "iam:TagRole",
                        "iam:PutRolePolicy",
                        "iam:DeleteRole"
                    ],
                    resources=["*"]
                )
            ]
        )

        # Attach the inline policy to your role
        role.attach_inline_policy(custom_policy)
        
        # Assume vpc, machine_image, and role are defined above
        if env_type != "prod":
            # Machine image (update with your AMI ID)
            machine_image = ec2.MachineImage.generic_linux({
                "ap-southeast-1": "ami-0220d4db08a04baa8"
            })

            # EC2 Instance in private subnet, no public IP
            instance = ec2.Instance(
                self,
                "BlinkMLEC2Instance",
                instance_type=ec2.InstanceType("r6i.2xlarge"),
                machine_image=machine_image,
                vpc=vpc,
                role=role,
                vpc_subnets=ec2.SubnetSelection(subnet_type=ec2.SubnetType.PRIVATE_WITH_EGRESS)
            )
        
        #generalML container
        asset = ecr_assets.DockerImageAsset(
            self, "GeneralMLDockerAsset",
            directory=str(Path(__file__).resolve().parent.parent.parent / "container" / "general_ml_container")
        )
        # Create shared batch infra for general_ml_container
        self.batch_setup = FargateBatchSetup(self, vpc=vpc, image_uri=asset.image_uri)

        # general_data_container
        data_asset = ecr_assets.DockerImageAsset(
            self, "GeneralDataDockerAsset",
            directory=str(Path(__file__).resolve().parent.parent.parent / "container" / "general_data_container")
        )
        #print the image URI for debugging
        print(f"General Data Container Image URI: {data_asset.image_uri}")

        # Lambda that can submit to general-purpose job
        submit_lambda = BatchSubmitterLambdaConstruct(
            self, "SubmitterLambda",
            job_queue=self.batch_setup.job_queue.ref,
            job_definition=self.batch_setup.job_def.ref
        )
        
        self.submit_lambda = submit_lambda.lambda_fn